Presentado por: Diego Quisi


# Entornos de soporte al desarrollo de simulaciones: Simpy - Parte 1

## 1. Instalación de Simpy
Existen diversas alternativas para instalar **simpy** (no confundir con _sympy_), sin embargo, en esta subsección mostraremos cómo hacerlo.


### 1.1. Pasos a seguir:
En virtud de que **simpy** requiere la versión 3.7 de Python, llevaremos a cabo el proceso para instalar dicha versión:
1. Creamos el entorno virtual llamado "simulacion" con la versión 3.4 de Python:

        conda activate
        
2. Installamos **simpy**  en el entorno creado:
        pip3 install -U simpy
        
3. Una vez que hemos arrancado el servidor de Jupyter (con el comando _jupyter notebook_) y tenemos un cuaderno abierto, podemos escoger con qué **kernel** trabajar:


Si la instalación ha sido exitosa, podremos ejecutar el siguiente código sin inconvenientes:

In [4]:
import simpy

# Definimos un reloj:
def reloj(env, nombre, tiempo):
    while True:
        print(nombre,">",env.now) 
        yield env.timeout(tiempo) # Pasamos el control al programa principal
        
env=simpy.Environment() # Creamos un entorno de simulación
env.process(reloj(env, 'R. Rapido',0.235)) # Ejecutamos un "reloj" rapido (timer) en el entorno con el tiempo especificado
env.process(reloj(env, 'R. Lento',0.47)) # Ejecutamos un "reloj" lento (timer) en el entorno con el tiempo especificado

env.run(until=2) # Ejecutamos los procesos 2 unidades de tiempo

R. Rapido > 0
R. Lento > 0
R. Rapido > 0.235
R. Lento > 0.47
R. Rapido > 0.47
R. Rapido > 0.705
R. Lento > 0.94
R. Rapido > 0.94
R. Rapido > 1.1749999999999998
R. Rapido > 1.4099999999999997
R. Lento > 1.41
R. Rapido > 1.6449999999999996
R. Rapido > 1.8799999999999994
R. Lento > 1.88


## 2. Ejemplo del negocio de lavado de autos [5]

Este ejemplo permite simular un negocio de lavado de automóviles. De igual forma, es importante destacar que este ejemplo aborda los siguientes puntos:

* Estados de espera por otros procesos
* Recursos: clase **Resource**


### 2.1. Características del sistema real a simular

Es importante observar que el negocio de lavado de autos tiene características propias a su naturaleza. A continuación establecemos dichas peculiaridades a tener en mente en el momento de llevar a cabo la simulación:

* El negocio tiene un número limitado de máquinas de lavado.
* Se puede recibir un cierto número de vehículos para ser lavados, de los cuáles los que no estén siendo procesados tendrán que esperar.
* Una vez que un vehículo entra a la máquina, debe ser lavado y solo al finalizar la limpieza podrá salir de la misma, dejando un espacio libre a otro vehículo.

### 2.2. Desarrollo de la simulación

A continuación procedemos a desarrollar la simulación.

Como primer paso, importamos las librerías necesarias y establecemos los parámetros requeridos:
* Variables de interés
* Tiempo de simulación
* Parámetros de inicialización (número de máquinas, tiempo de lavado, intervalo de llegada de vehículos).

In [5]:
import simpy
import random

# Maximo de vehiculos que puede recibir el negocio
MAX_VEHICULOS = 57 
# Total de maquinas de lavado con que cuenta el negocio
NUM_MAQUINAS = 3
# Tiempo que tarda en lavarse un vehiculo (minutos)
TIEMPO_LAVADO = 7 
# Intervalo de tiempo en que llegan vehiculos (minutos)
INTERVALO_LLEGADA = 9
# Tiempo de simulación
TIEMPO_SIMULACION = 23

class Lavanderia(object):
    
    def __init__(self, environment, num_maquinas, tiempo_lavado):
        # Guardamos como variable el entorno de ejecucion
        self.env=environment
        # Creamos el recurso que representa las maquinas
        self.maquinas = simpy.Resource(environment, num_maquinas)
        # Variable para el tiempo de lavado
        self.tiempo_lavado = tiempo_lavado
        
    def lavar_vehiculo(self, vehiculo):
        # Este metodo representa el proceso de lavado del vehículo.
        # Se ingresa el vehículo y se lava
        
        # Simulamos el tiempo que tarda en lavarse el vehiculo
        # Es importante notar que la instruccion "yield" es distinta de "sleep"
        # ya que esta ultima bloquea el hilo de ejecucion durante 't' unidades de tiempo,
        # mientras que 'yield' no bloquea el hilo de ejecucion, solo lo suspende mientras
        # el evento de 'lavado' se realice
        yield self.env.timeout(TIEMPO_LAVADO)
        
        # Simulamos que se ha limpiado parte (%) de la suciedad del vehiculo
        # Para el % generamos un entero entre 30 y 90
        print('Removido  {%d%%} suciedad vehiculo => %s ' \
              % (random.randint(30,90), vehiculo))
        
    
def llegada_vehiculo(env, nombre, lavanderia):
    # Usamos el reloj de la simulacion (env.now()) para indicar a la
    # hora que llega el vehiculo con el nombre pasado como parametro
    print('Llega vehiculo: %s a la hora %.2f.' % (nombre, env.now))
    
    # Especificamos que vamos a usar un recurso (Resource) que representa
    # la maquina de lavado
    with lavanderia.maquinas.request() as maquina:
        # Ocupamos la maquina de lavado
        yield maquina
        # Indicamos que vehiculo entra a la lavanderia
        print('Entra vehiculo a lavarse: %s a la hora %.2f.' % (nombre, env.now))
        # Procesamos la operacion de lavado
        yield env.process(lavanderia.lavar_vehiculo(nombre))
        # Una vez que termina la llamada con 'yield', se indica que se ha lavado el vehiculo
        print('Vehiculo [%s] lavado a las %.2f.' % (nombre, env.now))
    
    
def ejecutar_simulacion(env, num_maquinas, tiempo_lavado, intervalo):
    lavanderia=Lavanderia(env, num_maquinas, tiempo_lavado)
    # Creamos 5 llegadas de vehiculos iniciales
    for i in range(5):
        env.process(llegada_vehiculo(env, 'Vehiculo-%d'%(i+1),lavanderia))
    
    # Ejecutamos la simulacion
    while True:
        yield env.timeout(random.randint(intervalo-2, intervalo+2))
        i+=1
        # Mientras se lavan los vehiculos generamos mas vehiculos
        env.process(llegada_vehiculo(env,'Vehiculo-%d'%(i+1),lavanderia))

print('Lavanderia UPS')
# Inicializamos la semilla aleatoria
random.seed(77)

# Creamos el entorno de simulacion
env=simpy.Environment()
env.process(ejecutar_simulacion(env, NUM_MAQUINAS, TIEMPO_LAVADO, INTERVALO_LLEGADA))

# Ejecutamos el proceso durante el tiempo de simulacion
env.run(until = TIEMPO_SIMULACION)


Lavanderia UPS
Llega vehiculo: Vehiculo-1 a la hora 0.00.
Llega vehiculo: Vehiculo-2 a la hora 0.00.
Llega vehiculo: Vehiculo-3 a la hora 0.00.
Llega vehiculo: Vehiculo-4 a la hora 0.00.
Llega vehiculo: Vehiculo-5 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-1 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-2 a la hora 0.00.
Entra vehiculo a lavarse: Vehiculo-3 a la hora 0.00.
Removido  {50%} suciedad vehiculo => Vehiculo-1 
Removido  {42%} suciedad vehiculo => Vehiculo-2 
Removido  {45%} suciedad vehiculo => Vehiculo-3 
Vehiculo [Vehiculo-1] lavado a las 7.00.
Vehiculo [Vehiculo-2] lavado a las 7.00.
Vehiculo [Vehiculo-3] lavado a las 7.00.
Entra vehiculo a lavarse: Vehiculo-4 a la hora 7.00.
Entra vehiculo a lavarse: Vehiculo-5 a la hora 7.00.
Llega vehiculo: Vehiculo-6 a la hora 9.00.
Entra vehiculo a lavarse: Vehiculo-6 a la hora 9.00.
Removido  {82%} suciedad vehiculo => Vehiculo-4 
Removido  {90%} suciedad vehiculo => Vehiculo-5 
Vehiculo [Vehiculo-4] lavado a las 14.00.

## Práctica  - SimPy 

* Investigar el proceso de atención de un paciente diagnosticado con covid-19, ademas tiempos de ateción, recursos necesarios y tipos de pacientes.  
* Investigar en cada una de las provincias asignadas, el número de hospitales, camas, respiradores y personal medico, limpieza. 
* Finalmente el costo de atender a un paciente con COVID-19 y medicamentos. 

# Investigación Provincia de Tungurahua.  

EL **Ministerio de Salud Publica** recomienda los sigientes aspectos para el manejo inicial de pacientes con COVID-19.  

El personal de salud y el personal de apoyo que este asignado al paciente deberá aplicar precauciones estándar y precauciones de acuerdo a las vías de transmisión en todo momento durante el manejo del paciente.  

El personal de salud que brinde la atención debe cumplir con las normas y protocolos para manejo de pacientes críticos, en base a su evolución clínica y de las complicaciones que puedan aparecer en el paciente.  

No existe tratamiento específico ni vacuna para coronavirus, por lo que, todo caso debe ser manejado para lo cual se requiere cumplir con las siguientes recomendaciones:  

* Activación del Comité Técnico de Prevención y Control de Infecciones del establecimiento de salud para aplicar medidas de contención.
* Monitoreo y registro continuo de constantes vitales (frecuencia cardiaca, frecuencia respiratoria, temperatura y tensión arterial).
* Administración de líquidos intravenosos para mantenimiento del equilibrio hidroelectrolítico y balance hídrico.
* Mantenimiento de niveles adecuados de oxígeno y presión arterial.
* Administración de antipiréticos. No administrar ASA, antinflamatorios no esferoidales (AINES).
*  Tratamiento de otras infecciones y/o complicaciones según se presenten.

Los tipos de paciente son:
* Pacientes Sospechosos: Pacientes con infección respiratoria aguda grave y wue requieren hospitalización, Paciente con enfermedad respiratoria aguda leve y que no requiere hospitalización.
* Paciente Probable: Un caso sospechoso a quien realizaron pruebas para coronavirus (COVID-19) y no es concluyente el resultado de la prueba. 
* Paciente Confirmado: Caso sospechoso con confirmación de laboratorio de coronavirus (COVID-19), independientemente de los signos y síntomas clínicos.  

### Proceso de atención en el Hospital General Docente Ambato

En el Hospital se ingresan a los pacientes con sospecha de Covid-19 y les toman las muestras cuyos resultados llegan a los tres días posteriores. Aún si salen negativas, los enfermos se quedan ocho días en observación para descartar completamente que se trate de un caso positivo y se aplica el protocolo con cerco epidemiológico, según el director provincial del IESS.

Tungurahua cuenta con el **Hospital General Docente Ambato (HGDA)**, para la atencion de pacientes con posibles sintomas de COVID-19, el HGDA cuenta con **120 camas disponibles**, se adecuó el quinto piso exclusivamente para pacientes contagiados de Covid-19 y cuentan con una carpa para la atención exclusiva de pacientes con temas respiratorios.

En Ambato hay una capacidad instalada para atender a 24 pacientes con Covid-19 en las **Unidades de Cuidados Intensivos (UCI)**, tanto en el Hospital General Docente como en el Hospital del IESS. El Hospital Ambato cuenta con 18 camas con sus respectivos respiradores en la UCI, de las cuales 15 están ocupadas, mientras que el IESS tiene seis camas en la UCI y cinco ya están con pacientes.





## Referencias
[1] Matloff, N. (2008). Introduction to Discrete-Event Simulation and the SimPy Language.

[2] Team Simpy (2017). SimPy Documentation, Release 3.0.10, 2017. URL: https://media.readthedocs.org/pdf/simpy/latest/simpy.pdf 